In [8]:
import csv
import random
import statistics

In [9]:
def load_data(filepath):
    random.seed(42)
    patients = []
    
    try:
        with open(filepath, "r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                # This section will trigger ValueError if fields contain text/garbage
                clean_record = {
                    "patient_id": int(row["\ufeffSubject ID"]),
                    "age": random.randint(25, 70),
                    "temperature": float(row["Temperature reading 1"]),
                    "heart_rate": int(row["Heart rate reading 1"]),
                    "systolic_bp": int(row["Diastolic blood pressure reading 1"]),
                    "oxygen_saturation": int(row["Oxygen saturation reading 1"]),
                }
                patients.append(clean_record)
        return patients

    except FileNotFoundError:
        print(f"Error: The file at {filepath} was not found.")
        return []
    
    except ValueError as e:
        print(f"Error: Numeric field contains invalid data (garbage/text). Details: {e}")
        return []

In [10]:
# mean
def calculate_mean(values):
    return sum(values) / len(values)

#median
def calculate_median(values):
    return statistics.median(values)

#mode
def calculate_mode(values):
    return statistics.mode(values)

#outliers
def calculate_outliers(values):
    if len(values) <= 5:
        return []
    
    sorted_values = sorted(values)

    q1 = statistics.quantiles(sorted_values, n=4)[0]
    q3 = statistics.quantiles(sorted_values, n=4)[2]

    iqr = q3 - q1

    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)

    return [value for value in values if value < lower_bound or value > upper_bound]

In [11]:
def generate_summary_report(filepath):

    data = load_data(filepath)
    if not data:
        return
    
    # Extracting specific columns for analysis
    heart_rate_list = [dat['heart_rate'] for dat in data]
    temperature_list = [dat['temperature'] for dat in data]

    # Calculate Statistics
    hr_outliers = calculate_outliers(heart_rate_list)
    temp_outliers = calculate_outliers(temperature_list)
        
    print("=" * 50)
    print("       LIFE CARE CLINICS: VITAL REPORT   ")
    print("=" * 50)
    print(" ")
    print(f"Total patient analyzed: {len(data)}")
    
    #average heart rate to one decimal
    print(f"Average Patient's Heart Rate: {calculate_mean(heart_rate_list):.1f} BPM")
    #average temperature to one decimal
    print(f"Average Patient's Temperature: {calculate_mean(temperature_list):.1f} °C")

    print('-'* 50)

    flagged_patients = []
    for p in data:
        if p["heart_rate"] in hr_outliers or p["temperature"] in temp_outliers:
            flagged_patients.append(p["patient_id"])
            
    if flagged_patients:
        print(f"ALERT: High-Risk Outliers detected!")
        print(f"Patient IDs requiring review: {list(set(flagged_patients))}")
    else:
        print("All patient vitals within normal statistical ranges.")
    
    print("=" * 50)

In [6]:
generate_summary_report("patient_vitals.csv")

       LIFE CARE CLINICS: VITAL REPORT   
 
Total patient analyzed: 102
Average Patient's Heart Rate: 82.2 BPM
Average Patient's Temperature: 36.2 °C
--------------------------------------------------
All patient vitals within normal statistical ranges.


In [7]:
generate_summary_report("patient_vitals.csv")

       LIFE CARE CLINICS: VITAL REPORT   
 
Total patient analyzed: 102
Average Patient's Heart Rate: 82.2 BPM
Average Patient's Temperature: 36.2 °C
--------------------------------------------------
All patient vitals within normal statistical ranges.
